Developer: Tejas Yuvraj Kamble B066
Email Id: tejask0512@gmail.com  

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

import warnings
warnings.filterwarnings('ignore')

In [ ]:
house=pd.read_csv("housing_data.csv")
house.head()

FileNotFoundError: ignored

In [ ]:
house.describe()

In [ ]:
house.isnull().sum()

In [ ]:
house.head()

In [ ]:
house.info()

In [ ]:
house

In [ ]:
house.columns

In [ ]:
req_columns=['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath','PoolArea','GarageArea','OpenPorchSF','EnclosedPorch','BedroomAbvGr', 'TotRmsAbvGrd','SalePrice']

In [ ]:
house=house[req_columns]
house

In [ ]:
len(house.columns)

In [ ]:
house_columns=house.columns
house_columns

In [ ]:
predictors=house[house_columns[house_columns!='SalePrice']]
predictors

In [ ]:
target=house['SalePrice']
target

In [ ]:
import seaborn as sns
sns.distplot(house.SalePrice)

In [ ]:
sns.scatterplot(x="LotArea",y="SalePrice",data=house)

In [ ]:
sns.scatterplot(x="GarageArea",y="SalePrice",data=house)

In [ ]:
sns.boxplot(data=house.SalePrice)

In [ ]:
correlation=house.corr()
correlation.loc["SalePrice"]

In [ ]:
import matplotlib.pyplot as plt
fig,axes=plt.subplots(figsize=(15,12))
sns.heatmap(correlation,square=True,annot=True,cmap='ocean')  

#Color Combinations for cmap:
#'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG',
#'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens',
#'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1',
#'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket', 'rocket_r', 'seismic',
#'seismic_r', 'spring', 'spring_r'

# Using Neural Network

In [ ]:
ncols=predictors.shape[1]

In [ ]:
import keras 
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
def neural_regression_model():
    model=Sequential()
    model.add(Dense(64,activation='relu',input_shape=(ncols,)))
    model.add(Dense(32,activation='relu'))
    
    model.add(Dense(1,activation='linear'))
    optimizer = keras.optimizers.Adam(lr=0.01)
    model.compile(optimizer=optimizer,loss='mean_squared_error')
    return model
    

### Using train test split for splitting on normalized data

In [ ]:
from sklearn.model_selection import train_test_split



X_train,X_test,y_train,y_test=train_test_split(predictors,target,test_size=0.3,random_state=10)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Scaling Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
model=neural_regression_model()
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200)
model.evaluate(X_test,y_test)


"we can change hyperparameters like learning rate, no of hidden layers and epoch value for better result from the model defination above"

In [ ]:
losses = pd.DataFrame(history.history)

plt.figure(figsize=(15,5))
sns.lineplot(data=losses,lw=3)
plt.xlabel('Epochs')
plt.ylabel('')
plt.title('Training Loss per Epoch')
sns.despine()

In [ ]:
plt.figure(figsize=(15,5))
sns.lineplot(data=losses,lw=3)
plt.xlabel("Epochs")
plt.ylabel("val loss")
plt.title("validation loss per epoch")
plt.show()

In [ ]:
predictions=model.predict(X_test)
predictions

In [ ]:
loss=history.history['loss']
loss

In [ ]:
#now we can calculate errors using
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score 


MAE=mean_absolute_error(y_pred=predictions,y_true=y_test)
print(f"Mean absolute error : {MAE}")
      
MSE=mean_squared_error(y_pred=predictions,y_true=y_test)
print(f"Mean Squared error : {MSE}")

RMSE=np.sqrt(mean_squared_error(y_pred=predictions,y_true=y_test))
print(f"Root mean squared error: {RMSE}")

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(y_test,predictions)
plt.plot(y_test,y_test,'r')
plt.show()

In [ ]:
#prediction of price for individual house
print(f"Orignal Price: {y_test.iloc[5]}")
print(f"Predicted Price : {predictions[5][0]}")

## Adding New house Data To Dataset

###  To add new  house data just change the below list of values w.r.t column name

In [ ]:
house.loc[len(house)]=[7475,1969,866,457,2,1,499,48,104,2,6,np.nan]
house.tail()

In [ ]:
house.shape

In [ ]:
new_house=house.drop("SalePrice",axis=1).iloc[-1]
print(new_house)
new_house=scaler.transform(new_house.values.reshape(-1,11))
model.predict(new_house)[0,0]

## New house Data with its Predicted SalePrice

### Predicted SalePrice will be added to the Dataset 

In [ ]:
house['SalePrice'].iloc[-1]=model.predict(new_house)[0,0]
house.tail()